# 🌳 Notebook 2 : Raisonnement et Logique Avancés

## Tree-of-Thoughts, Self-Consistency et Raisonnement Complexe

Dans ce notebook, nous explorons les techniques de raisonnement avancées qui permettent aux LLMs de résoudre des problèmes complexes en explorant plusieurs chemins de pensée.

### 🎯 Objectifs d'apprentissage :
- Maîtriser Tree-of-Thoughts (ToT) pour l'exploration multi-chemins
- Implémenter Self-Consistency pour améliorer la fiabilité
- Utiliser Step-Back Prompting pour l'abstraction
- Créer des systèmes de raisonnement robustes
- Évaluer et sélectionner les meilleurs chemins de pensée

## 📚 Configuration et Imports

In [6]:
import os
import json
import time
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

# Configuration des APIs
import openai
from anthropic import Anthropic
import google.generativeai as genai

# Configuration des clients
openai.api_key = os.getenv('OPENAI_API_KEY')
anthropic = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

print("✅ Configuration terminée !")

✅ Configuration terminée !


## 1. 🌳 Tree-of-Thoughts (ToT)

### Qu'est-ce que Tree-of-Thoughts ?

Tree-of-Thoughts est une technique qui permet aux LLMs d'explorer plusieurs chemins de raisonnement en parallèle, d'évaluer chaque branche, et de sélectionner le meilleur chemin.

**Avantages :**
- 🎯 Exploration exhaustive des solutions
- 📊 Auto-évaluation des chemins
- 🔄 Possibilité de revenir en arrière
- 💡 Découverte de solutions créatives

In [7]:
@dataclass
class ThoughtNode:
    """Représente un nœud dans l'arbre de pensées"""
    content: str
    score: float
    depth: int
    parent: Optional['ThoughtNode'] = None
    children: List['ThoughtNode'] = None
    
    def __post_init__(self):
        if self.children is None:
            self.children = []

class TreeOfThoughts:
    """Implémentation de Tree-of-Thoughts"""
    
    def __init__(self, model: str = "gpt-4", branches: int = 3, depth: int = 3):
        self.model = model
        self.branches = branches
        self.max_depth = depth
        self.root = None
    
    def generate_thoughts(self, prompt: str, context: str = "") -> List[str]:
        """Génère plusieurs pensées/approches pour un problème"""
        generation_prompt = f"""
        Problème : {prompt}
        {f'Contexte : {context}' if context else ''}
        
        Génère {self.branches} approches différentes pour résoudre ce problème.
        Pour chaque approche :
        1. Donne un titre court
        2. Explique la logique en 2-3 phrases
        3. Liste les étapes principales
        
        Format : 
        Approche 1: [Titre]
        [Explication et étapes]
        ---
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": generation_prompt}],
                temperature=0.8,
                n=1
            )
            
            thoughts_text = response.choices[0].message.content
            thoughts = thoughts_text.split('---')
            return [t.strip() for t in thoughts if t.strip()]
            
        except Exception as e:
            print(f"Erreur génération : {e}")
            return []
    
    def evaluate_thought(self, thought: str, criteria: List[str]) -> float:
        """Évalue une pensée selon des critères"""
        eval_prompt = f"""
        Évalue cette approche selon les critères suivants :
        
        Approche : {thought}
        
        Critères d'évaluation :
        {chr(10).join(f'- {c}' for c in criteria)}
        
        Pour chaque critère, donne une note sur 10.
        Puis calcule la moyenne pondérée.
        
        Format de réponse :
        - [Critère 1]: X/10 - [Justification]
        - [Critère 2]: Y/10 - [Justification]
        ...
        Score final : Z/10
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": eval_prompt}],
                temperature=0.3
            )
            
            result = response.choices[0].message.content
            # Extraire le score final
            if "Score final" in result:
                score_text = result.split("Score final")[-1]
                score = float(score_text.split("/")[0].replace(":", "").strip())
                return score / 10.0
            return 0.5
            
        except Exception as e:
            print(f"Erreur évaluation : {e}")
            return 0.5
    
    def expand_node(self, node: ThoughtNode, problem: str) -> List[ThoughtNode]:
        """Expand un nœud en générant des sous-pensées"""
        if node.depth >= self.max_depth:
            return []
        
        expansion_prompt = f"""
        Problème initial : {problem}
        
        Approche actuelle : {node.content}
        
        Développe cette approche en {self.branches} sous-étapes ou raffinements.
        Chaque sous-étape doit être plus spécifique et actionnable.
        """
        
        sub_thoughts = self.generate_thoughts(expansion_prompt)
        children = []
        
        for thought in sub_thoughts[:self.branches]:
            child = ThoughtNode(
                content=thought,
                score=0,  # Sera évalué plus tard
                depth=node.depth + 1,
                parent=node
            )
            children.append(child)
            node.children.append(child)
        
        return children
    
    def solve(self, problem: str, criteria: List[str]) -> Dict:
        """Résout un problème en utilisant Tree-of-Thoughts"""
        print(f"🌳 Démarrage Tree-of-Thoughts pour : {problem[:50]}...")
        
        # Générer les pensées initiales
        initial_thoughts = self.generate_thoughts(problem)
        
        # Créer le nœud racine
        self.root = ThoughtNode(content="Root", score=0, depth=0)
        
        # Créer et évaluer les nœuds de premier niveau
        for thought in initial_thoughts[:self.branches]:
            node = ThoughtNode(
                content=thought,
                score=self.evaluate_thought(thought, criteria),
                depth=1,
                parent=self.root
            )
            self.root.children.append(node)
        
        # Sélectionner les meilleurs nœuds pour expansion
        nodes_to_expand = sorted(self.root.children, key=lambda x: x.score, reverse=True)[:2]
        
        # Expansion récursive
        for node in nodes_to_expand:
            if node.score > 0.7:  # Seuil de qualité
                children = self.expand_node(node, problem)
                for child in children:
                    child.score = self.evaluate_thought(child.content, criteria)
        
        # Trouver le meilleur chemin
        best_path = self.find_best_path()
        
        return {
            'best_path': best_path,
            'tree_stats': self.get_tree_stats(),
            'all_thoughts': self.get_all_thoughts()
        }
    
    def find_best_path(self) -> List[ThoughtNode]:
        """Trouve le meilleur chemin dans l'arbre"""
        def get_paths(node: ThoughtNode, current_path: List[ThoughtNode]) -> List[List[ThoughtNode]]:
            if not node.children:
                return [current_path + [node]]
            
            paths = []
            for child in node.children:
                paths.extend(get_paths(child, current_path + [node]))
            return paths
        
        all_paths = get_paths(self.root, [])
        
        # Calculer le score moyen de chaque chemin
        path_scores = []
        for path in all_paths:
            if len(path) > 1:  # Ignorer le root
                avg_score = np.mean([n.score for n in path[1:]])
                path_scores.append((path, avg_score))
        
        # Retourner le meilleur chemin
        if path_scores:
            best_path, _ = max(path_scores, key=lambda x: x[1])
            return best_path[1:]  # Exclure le root
        return []
    
    def get_tree_stats(self) -> Dict:
        """Obtient des statistiques sur l'arbre"""
        def count_nodes(node: ThoughtNode) -> int:
            return 1 + sum(count_nodes(child) for child in node.children)
        
        total_nodes = count_nodes(self.root) - 1  # Exclure root
        all_scores = []
        
        def collect_scores(node: ThoughtNode):
            if node != self.root:
                all_scores.append(node.score)
            for child in node.children:
                collect_scores(child)
        
        collect_scores(self.root)
        
        return {
            'total_nodes': total_nodes,
            'avg_score': np.mean(all_scores) if all_scores else 0,
            'max_score': max(all_scores) if all_scores else 0,
            'min_score': min(all_scores) if all_scores else 0
        }
    
    def get_all_thoughts(self) -> List[Dict]:
        """Récupère toutes les pensées avec leurs scores"""
        thoughts = []
        
        def collect_thoughts(node: ThoughtNode, path: str = ""):
            if node != self.root:
                thoughts.append({
                    'content': node.content,
                    'score': node.score,
                    'depth': node.depth,
                    'path': path
                })
            
            for i, child in enumerate(node.children):
                new_path = f"{path}/{i}" if path else str(i)
                collect_thoughts(child, new_path)
        
        collect_thoughts(self.root)
        return sorted(thoughts, key=lambda x: x['score'], reverse=True)

### 🎯 Exemple pratique : Résolution de problème complexe avec ToT

In [8]:
# Problème complexe à résoudre
problem = """
Une startup EdTech veut augmenter son taux de rétention utilisateur de 40% à 70% en 6 mois.
Contraintes : budget limité (50k€), équipe de 5 personnes, app mobile existante.
"""

# Critères d'évaluation
criteria = [
    "Faisabilité avec les contraintes données",
    "Impact potentiel sur la rétention",
    "Rapidité de mise en œuvre",
    "Coût-efficacité",
    "Mesurabilité des résultats"
]

# Créer et exécuter ToT
tot = TreeOfThoughts(model="gpt-3.5-turbo", branches=3, depth=2)
solution = tot.solve(problem, criteria)

# Afficher les résultats
print("\n🌟 Meilleur chemin de solution :")
for i, node in enumerate(solution['best_path']):
    print(f"\n{'  ' * i}Niveau {i+1} (Score: {node.score:.2f})")
    print(f"{'  ' * i}{node.content[:200]}...")

print(f"\n📊 Statistiques de l'arbre :")
stats = solution['tree_stats']
print(f"- Nœuds explorés : {stats['total_nodes']}")
print(f"- Score moyen : {stats['avg_score']:.2f}")
print(f"- Score max : {stats['max_score']:.2f}")

🌳 Démarrage Tree-of-Thoughts pour : 
Une startup EdTech veut augmenter son taux de rét...
Erreur génération : 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


🌟 Meilleur chemin de solution :

📊 Statistiques de l'arbre :
- Nœuds explorés : 0
- Score moyen : 0.00
- Score max : 0.00


## 2. 🔄 Self-Consistency

### Principe de Self-Consistency

Self-Consistency génère plusieurs réponses indépendantes à la même question, puis analyse les convergences pour identifier la réponse la plus fiable.

**Avantages :**
- ✅ Réduit les erreurs aléatoires
- 🎯 Augmente la fiabilité
- 📊 Permet d'identifier l'incertitude
- 🔍 Révèle différentes perspectives

In [9]:
class SelfConsistency:
    """Implémentation de Self-Consistency pour améliorer la fiabilité"""
    
    def __init__(self, model: str = "gpt-3.5-turbo", samples: int = 5):
        self.model = model
        self.samples = samples
    
    def generate_responses(self, prompt: str, variations: bool = True) -> List[str]:
        """Génère plusieurs réponses au même prompt"""
        responses = []
        
        # Variations du prompt pour plus de diversité
        prompts = []
        if variations:
            base_variations = [
                prompt,
                f"Réfléchis étape par étape : {prompt}",
                f"En considérant tous les aspects : {prompt}",
                f"Analyse en profondeur : {prompt}",
                f"Donne ta meilleure réponse : {prompt}"
            ]
            prompts = base_variations[:self.samples]
        else:
            prompts = [prompt] * self.samples
        
        # Générer les réponses en parallèle
        with ThreadPoolExecutor(max_workers=3) as executor:
            futures = []
            for i, p in enumerate(prompts):
                future = executor.submit(self._generate_single, p, 0.7 + i * 0.05)
                futures.append(future)
            
            for future in futures:
                try:
                    response = future.result(timeout=30)
                    if response:
                        responses.append(response)
                except Exception as e:
                    print(f"Erreur génération : {e}")
        
        return responses
    
    def _generate_single(self, prompt: str, temperature: float) -> str:
        """Génère une seule réponse"""
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=temperature,
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Erreur API : {e}")
            return ""
    
    def extract_key_elements(self, response: str) -> Dict:
        """Extrait les éléments clés d'une réponse"""
        extraction_prompt = f"""
        Analyse cette réponse et extrais :
        1. Les points principaux (liste)
        2. Les chiffres/données mentionnés
        3. Les recommandations spécifiques
        4. Le ton/sentiment général
        
        Réponse à analyser :
        {response}
        
        Format JSON requis.
        """
        
        try:
            result = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": extraction_prompt}],
                temperature=0.3
            )
            
            # Parser le JSON
            content = result.choices[0].message.content
            # Extraire le JSON même s'il est entouré de texte
            import re
            json_match = re.search(r'\{[^{}]*\}', content, re.DOTALL)
            if json_match:
                return json.loads(json_match.group())
            return {}
            
        except Exception as e:
            print(f"Erreur extraction : {e}")
            return {}
    
    def analyze_consistency(self, responses: List[str]) -> Dict:
        """Analyse la cohérence entre les réponses"""
        if not responses:
            return {'error': 'Aucune réponse à analyser'}
        
        # Extraire les éléments clés de chaque réponse
        all_elements = []
        for resp in responses:
            elements = self.extract_key_elements(resp)
            if elements:
                all_elements.append(elements)
        
        # Analyser les convergences
        consistency_prompt = f"""
        Analyse la cohérence entre ces {len(responses)} réponses :
        
        {chr(10).join([f'Réponse {i+1}: {r[:200]}...' for i, r in enumerate(responses)])}
        
        Identifie :
        1. Points de convergence (mentionnés dans 3+ réponses)
        2. Points de divergence majeurs
        3. Niveau de cohérence global (0-100%)
        4. Recommandation synthétisée
        """
        
        try:
            analysis = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": consistency_prompt}],
                temperature=0.3
            )
            
            return {
                'analysis': analysis.choices[0].message.content,
                'num_responses': len(responses),
                'elements_extracted': len(all_elements)
            }
            
        except Exception as e:
            print(f"Erreur analyse : {e}")
            return {'error': str(e)}
    
    def get_consensus(self, prompt: str, return_all: bool = False) -> Dict:
        """Obtient un consensus via Self-Consistency"""
        print(f"🔄 Génération de {self.samples} réponses...")
        
        # Générer plusieurs réponses
        responses = self.generate_responses(prompt)
        
        if not responses:
            return {'error': 'Aucune réponse générée'}
        
        print(f"✅ {len(responses)} réponses générées")
        
        # Analyser la cohérence
        consistency = self.analyze_consistency(responses)
        
        # Créer une réponse consensus
        consensus_prompt = f"""
        Basé sur ces {len(responses)} réponses différentes au même problème,
        crée une réponse consensus qui intègre les meilleurs éléments de chaque réponse.
        
        Problème original : {prompt}
        
        Réponses à synthétiser :
        {chr(10).join([f'--- Réponse {i+1} ---\n{r}' for i, r in enumerate(responses)])}
        
        Crée une réponse finale optimale qui :
        - Intègre les points de convergence
        - Résout les contradictions
        - Offre la solution la plus complète et fiable
        """
        
        try:
            consensus_response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": consensus_prompt}],
                temperature=0.3,
                max_tokens=1500
            )
            
            result = {
                'consensus': consensus_response.choices[0].message.content,
                'consistency_analysis': consistency,
                'num_responses': len(responses)
            }
            
            if return_all:
                result['all_responses'] = responses
            
            return result
            
        except Exception as e:
            print(f"Erreur consensus : {e}")
            return {'error': str(e)}

### 🎯 Exemple : Décision critique avec Self-Consistency

In [10]:
# Question nécessitant une haute fiabilité
critical_question = """
Notre entreprise SaaS B2B (CRM pour PME) stagne à 2M€ ARR depuis 6 mois.
Nous hésitons entre 3 stratégies :
1. Monter en gamme vers l'entreprise (investissement 500k€)
2. Expansion internationale (investissement 300k€)
3. Ajouter des features IA (investissement 400k€)

Quelle stratégie recommandes-tu et pourquoi ?
"""

# Utiliser Self-Consistency
sc = SelfConsistency(samples=5)
result = sc.get_consensus(critical_question, return_all=True)

# Afficher l'analyse
print("\n📊 Analyse de cohérence :")
print(result['consistency_analysis']['analysis'])

print("\n✅ Réponse consensus :")
print(result['consensus'])

# Optionnel : voir toutes les réponses
if 'all_responses' in result:
    print(f"\n🔍 Aperçu des {len(result['all_responses'])} réponses générées :")
    for i, resp in enumerate(result['all_responses']):
        print(f"\nRéponse {i+1} (extrait) : {resp[:150]}...")

🔄 Génération de 5 réponses...
Erreur API : 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Erreur API : 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Erreur API

KeyError: 'consistency_analysis'

## 3. 🔙 Step-Back Prompting

### Principe du Step-Back Prompting

Step-Back Prompting résout les problèmes en prenant d'abord du recul pour comprendre les principes fondamentaux avant de s'attaquer au cas spécifique.

**Processus :**
1. 🎯 Identifier la question spécifique
2. 🔍 Formuler une question plus générale
3. 📚 Comprendre les principes fondamentaux
4. 🔧 Appliquer au cas spécifique

In [ ]:
class StepBackPrompting:
    """Implémentation de Step-Back Prompting pour la résolution abstraite"""
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
    
    def generate_abstract_question(self, specific_question: str) -> str:
        """Génère une question plus abstraite/générale"""
        prompt = f"""
        Question spécifique : {specific_question}
        
        Reformule cette question de manière plus abstraite et générale.
        La question abstraite doit :
        1. Capturer l'essence du problème
        2. Être applicable à une classe plus large de situations
        3. Permettre d'identifier les principes fondamentaux
        
        Exemples :
        - Spécifique : "Comment augmenter les ventes de mon restaurant italien ?"
        - Abstraite : "Quels sont les principes pour augmenter les revenus d'un business de service local ?"
        
        Question abstraite :
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Erreur : {e}")
            return ""
    
    def get_fundamental_principles(self, abstract_question: str) -> str:
        """Obtient les principes fondamentaux pour la question abstraite"""
        prompt = f"""
        Question : {abstract_question}
        
        Identifie et explique les principes fondamentaux qui s'appliquent.
        Pour chaque principe :
        1. Nom du principe
        2. Explication claire
        3. Pourquoi c'est important
        4. Exemples d'application
        
        Vise 3-5 principes fondamentaux.
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.5,
                max_tokens=1000
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Erreur : {e}")
            return ""
    
    def apply_to_specific(self, principles: str, specific_question: str, context: str = "") -> str:
        """Applique les principes au cas spécifique"""
        prompt = f"""
        Principes fondamentaux identifiés :
        {principles}
        
        Question spécifique à résoudre :
        {specific_question}
        
        {f'Contexte additionnel : {context}' if context else ''}
        
        Applique ces principes pour créer une solution spécifique et actionnable.
        Structure ta réponse :
        1. Rappel des principes pertinents
        2. Application concrète au cas
        3. Plan d'action détaillé
        4. Métriques de succès
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.6,
                max_tokens=1500
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Erreur : {e}")
            return ""
    
    def solve_with_stepback(self, specific_question: str, context: str = "") -> Dict:
        """Pipeline complet de Step-Back Prompting"""
        print("🔙 Démarrage Step-Back Prompting...")
        
        # Étape 1 : Générer question abstraite
        print("1️⃣ Génération de la question abstraite...")
        abstract_q = self.generate_abstract_question(specific_question)
        
        if not abstract_q:
            return {'error': 'Impossible de générer la question abstraite'}
        
        # Étape 2 : Obtenir les principes
        print("2️⃣ Identification des principes fondamentaux...")
        principles = self.get_fundamental_principles(abstract_q)
        
        if not principles:
            return {'error': 'Impossible d\'identifier les principes'}
        
        # Étape 3 : Appliquer au cas spécifique
        print("3️⃣ Application au cas spécifique...")
        solution = self.apply_to_specific(principles, specific_question, context)
        
        return {
            'specific_question': specific_question,
            'abstract_question': abstract_q,
            'principles': principles,
            'solution': solution
        }
    
    def compare_with_direct(self, question: str) -> Dict:
        """Compare Step-Back avec approche directe"""
        print("🔬 Comparaison Step-Back vs Direct...")
        
        # Approche directe
        direct_prompt = f"Réponds directement : {question}"
        try:
            direct_response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": direct_prompt}],
                temperature=0.7
            )
            direct_answer = direct_response.choices[0].message.content
        except:
            direct_answer = "Erreur"
        
        # Approche Step-Back
        stepback_result = self.solve_with_stepback(question)
        
        return {
            'direct_approach': direct_answer,
            'stepback_approach': stepback_result
        }

### 🎯 Exemple : Résolution complexe avec Step-Back

In [ ]:
# Problème spécifique complexe
specific_problem = """
Notre plateforme de e-learning pour développeurs a un problème :
70% des utilisateurs abandonnent après le premier cours.
Comment améliorer drastiquement l'engagement ?
"""

context = """
- Plateforme lancée il y a 6 mois
- 5000 utilisateurs inscrits
- Cours de 2-4 heures en moyenne
- Principalement du contenu vidéo
- Audience : développeurs juniors à mid-level
"""

# Utiliser Step-Back Prompting
sb = StepBackPrompting()
result = sb.solve_with_stepback(specific_problem, context)

# Afficher les résultats
print("\n❓ Question abstraite générée :")
print(result['abstract_question'])

print("\n📚 Principes fondamentaux identifiés :")
print(result['principles'][:500] + "...")

print("\n✅ Solution spécifique :")
print(result['solution'])

### 🔬 Comparaison : Step-Back vs Approche Directe

In [ ]:
# Comparer les deux approches
comparison = sb.compare_with_direct(specific_problem)

print("🔄 APPROCHE DIRECTE :")
print(comparison['direct_approach'][:500] + "...")

print("\n\n🔙 APPROCHE STEP-BACK :")
if 'solution' in comparison['stepback_approach']:
    print(comparison['stepback_approach']['solution'][:500] + "...")

print("\n\n📊 Analyse comparative :")
print("- L'approche directe donne des solutions immédiates mais parfois superficielles")
print("- Step-Back identifie les causes profondes et propose des solutions systémiques")
print("- Step-Back est plus adapté aux problèmes complexes nécessitant une compréhension profonde")

## 4. 🧩 Combinaison des Techniques

### Framework de Raisonnement Hybride

Combinons ToT, Self-Consistency et Step-Back pour créer un système de raisonnement ultra-robuste.

In [ ]:
class HybridReasoning:
    """Combine plusieurs techniques de raisonnement avancées"""
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.tot = TreeOfThoughts(model, branches=3, depth=2)
        self.sc = SelfConsistency(model, samples=3)
        self.sb = StepBackPrompting(model)
    
    def analyze_problem_complexity(self, problem: str) -> Dict:
        """Analyse la complexité du problème pour choisir la meilleure approche"""
        analysis_prompt = f"""
        Analyse ce problème et détermine :
        1. Niveau de complexité (1-10)
        2. Type de problème (analytique, créatif, stratégique, technique)
        3. Besoin de fiabilité (1-10)
        4. Besoin d'exploration (1-10)
        
        Problème : {problem}
        
        Réponds en JSON.
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": analysis_prompt}],
                temperature=0.3
            )
            
            # Parser la réponse
            import re
            content = response.choices[0].message.content
            # Essayer d'extraire les valeurs même si ce n'est pas du JSON parfait
            
            complexity = int(re.search(r'complexité.*?(\d+)', content, re.I).group(1)) if re.search(r'complexité.*?(\d+)', content, re.I) else 5
            reliability = int(re.search(r'fiabilité.*?(\d+)', content, re.I).group(1)) if re.search(r'fiabilité.*?(\d+)', content, re.I) else 5
            exploration = int(re.search(r'exploration.*?(\d+)', content, re.I).group(1)) if re.search(r'exploration.*?(\d+)', content, re.I) else 5
            
            return {
                'complexity': complexity,
                'reliability_need': reliability,
                'exploration_need': exploration
            }
            
        except Exception as e:
            print(f"Erreur analyse : {e}")
            return {'complexity': 5, 'reliability_need': 5, 'exploration_need': 5}
    
    def select_techniques(self, analysis: Dict) -> List[str]:
        """Sélectionne les techniques appropriées selon l'analyse"""
        techniques = []
        
        # Step-Back pour les problèmes complexes
        if analysis['complexity'] >= 7:
            techniques.append('step_back')
        
        # Self-Consistency pour la fiabilité
        if analysis['reliability_need'] >= 7:
            techniques.append('self_consistency')
        
        # Tree-of-Thoughts pour l'exploration
        if analysis['exploration_need'] >= 6:
            techniques.append('tree_of_thoughts')
        
        # Toujours au moins une technique
        if not techniques:
            techniques.append('self_consistency')
        
        return techniques
    
    def solve_hybrid(self, problem: str, context: str = "") -> Dict:
        """Résout un problème en utilisant la meilleure combinaison de techniques"""
        print("🧩 Démarrage du raisonnement hybride...")
        
        # Analyser le problème
        print("📊 Analyse du problème...")
        analysis = self.analyze_problem_complexity(problem)
        print(f"Complexité : {analysis['complexity']}/10")
        print(f"Besoin fiabilité : {analysis['reliability_need']}/10")
        print(f"Besoin exploration : {analysis['exploration_need']}/10")
        
        # Sélectionner les techniques
        techniques = self.select_techniques(analysis)
        print(f"\n🔧 Techniques sélectionnées : {', '.join(techniques)}")
        
        results = {}
        
        # Appliquer Step-Back si nécessaire
        if 'step_back' in techniques:
            print("\n🔙 Application Step-Back...")
            sb_result = self.sb.solve_with_stepback(problem, context)
            results['step_back'] = sb_result
            
            # Utiliser les principes pour enrichir le contexte
            if 'principles' in sb_result:
                context += f"\n\nPrincipes identifiés : {sb_result['principles'][:200]}"
        
        # Appliquer Tree-of-Thoughts si nécessaire
        if 'tree_of_thoughts' in techniques:
            print("\n🌳 Application Tree-of-Thoughts...")
            criteria = [
                "Faisabilité pratique",
                "Impact potentiel",
                "Innovation de l'approche",
                "Risques et mitigation"
            ]
            tot_result = self.tot.solve(problem + "\n" + context, criteria)
            results['tree_of_thoughts'] = tot_result
        
        # Appliquer Self-Consistency si nécessaire
        if 'self_consistency' in techniques:
            print("\n🔄 Application Self-Consistency...")
            
            # Créer un prompt enrichi avec les résultats précédents
            enriched_prompt = problem
            if 'step_back' in results and 'solution' in results['step_back']:
                enriched_prompt += f"\n\nConsidère ces insights : {results['step_back']['solution'][:300]}..."
            if 'tree_of_thoughts' in results and results['tree_of_thoughts']['best_path']:
                best_thought = results['tree_of_thoughts']['best_path'][0].content
                enriched_prompt += f"\n\nApproche prometteuse identifiée : {best_thought[:200]}..."
            
            sc_result = self.sc.get_consensus(enriched_prompt)
            results['self_consistency'] = sc_result
        
        # Synthèse finale
        print("\n🎯 Synthèse des résultats...")
        final_synthesis = self.create_synthesis(problem, results, techniques)
        
        return {
            'analysis': analysis,
            'techniques_used': techniques,
            'results': results,
            'synthesis': final_synthesis
        }
    
    def create_synthesis(self, problem: str, results: Dict, techniques: List[str]) -> str:
        """Crée une synthèse finale des différentes approches"""
        synthesis_prompt = f"""
        Problème original : {problem}
        
        Résultats des différentes techniques de raisonnement :
        
        """
        
        if 'step_back' in results and 'solution' in results['step_back']:
            synthesis_prompt += f"\n**Step-Back Analysis:**\n{results['step_back']['solution'][:400]}...\n"
        
        if 'tree_of_thoughts' in results and results['tree_of_thoughts']['best_path']:
            best_thoughts = [n.content[:200] for n in results['tree_of_thoughts']['best_path'][:2]]
            synthesis_prompt += f"\n**Tree-of-Thoughts Best Path:**\n{chr(10).join(best_thoughts)}\n"
        
        if 'self_consistency' in results and 'consensus' in results['self_consistency']:
            synthesis_prompt += f"\n**Self-Consistency Consensus:**\n{results['self_consistency']['consensus'][:400]}...\n"
        
        synthesis_prompt += """
        
        Crée une synthèse finale qui :
        1. Intègre les meilleures idées de chaque approche
        2. Résout les contradictions éventuelles
        3. Propose un plan d'action clair et priorisé
        4. Identifie les risques et les mitigation
        5. Définit des métriques de succès
        """
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": synthesis_prompt}],
                temperature=0.4,
                max_tokens=1500
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Erreur synthèse : {e}")
            return "Erreur lors de la création de la synthèse"

### 🚀 Exemple ultime : Problème complexe avec raisonnement hybride

In [ ]:
# Problème très complexe nécessitant plusieurs approches
complex_problem = """
Notre entreprise FinTech (paiements B2B internationaux) fait face à une décision critique :
- Croissance ralentie (15% vs 50% l'an dernier)
- Nouveaux concurrents agressifs
- Réglementation changeante en Europe
- Opportunité d'acquisition d'un concurrent plus petit (15M€)
- Ou investir dans l'expansion Asie (20M€)
- Budget total disponible : 25M€

Quelle stratégie adopter pour retrouver une croissance forte tout en minimisant les risques ?
"""

additional_context = """
- 200 employés actuellement
- ARR : 30M€
- Présence : Europe (80%), US (20%)
- Technologie propriétaire forte
- NPS : 45 (en baisse)
"""

# Résoudre avec approche hybride
hybrid = HybridReasoning()
solution = hybrid.solve_hybrid(complex_problem, additional_context)

# Afficher la synthèse finale
print("\n" + "="*50)
print("🎯 SYNTHÈSE FINALE - RECOMMANDATION STRATÉGIQUE")
print("="*50)
print(solution['synthesis'])

# Statistiques sur l'analyse
print("\n📊 Statistiques de l'analyse :")
print(f"- Techniques utilisées : {', '.join(solution['techniques_used'])}")
print(f"- Complexité identifiée : {solution['analysis']['complexity']}/10")
if 'tree_of_thoughts' in solution['results']:
    stats = solution['results']['tree_of_thoughts']['tree_stats']
    print(f"- Chemins explorés (ToT) : {stats['total_nodes']}")
if 'self_consistency' in solution['results']:
    print(f"- Réponses générées (SC) : {solution['results']['self_consistency']['num_responses']}")

## 5. 📊 Métriques et Évaluation du Raisonnement

### Framework d'évaluation des techniques de raisonnement

In [ ]:
class ReasoningEvaluator:
    """Évalue et compare différentes techniques de raisonnement"""
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.metrics = {
            'coherence': 'Cohérence logique de la réponse',
            'completeness': 'Exhaustivité de la solution',
            'practicality': 'Applicabilité pratique',
            'innovation': 'Originalité et créativité',
            'clarity': 'Clarté et structure'
        }
    
    def evaluate_response(self, response: str, problem: str) -> Dict[str, float]:
        """Évalue une réponse selon plusieurs métriques"""
        scores = {}
        
        for metric, description in self.metrics.items():
            eval_prompt = f"""
            Problème : {problem}
            
            Réponse à évaluer : {response}
            
            Évalue cette réponse sur le critère : {description}
            Donne une note de 0 à 10 et justifie brièvement.
            
            Format : [Note]/10 - [Justification courte]
            """
            
            try:
                result = openai.ChatCompletion.create(
                    model=self.model,
                    messages=[{"role": "user", "content": eval_prompt}],
                    temperature=0.3
                )
                
                # Extraire la note
                content = result.choices[0].message.content
                import re
                match = re.search(r'(\d+(\.\d+)?)/10', content)
                if match:
                    scores[metric] = float(match.group(1))
                else:
                    scores[metric] = 5.0
                    
            except Exception as e:
                scores[metric] = 5.0
        
        return scores
    
    def compare_techniques(self, problem: str, responses: Dict[str, str]) -> Dict:
        """Compare plusieurs techniques sur le même problème"""
        results = {}
        
        for technique, response in responses.items():
            print(f"\n📊 Évaluation de {technique}...")
            scores = self.evaluate_response(response, problem)
            results[technique] = {
                'scores': scores,
                'average': np.mean(list(scores.values()))
            }
        
        # Créer un rapport comparatif
        report = self.generate_comparison_report(results)
        
        return {
            'detailed_scores': results,
            'report': report,
            'winner': max(results.items(), key=lambda x: x[1]['average'])[0]
        }
    
    def generate_comparison_report(self, results: Dict) -> str:
        """Génère un rapport de comparaison détaillé"""
        report = "📊 RAPPORT COMPARATIF DES TECHNIQUES\n"
        report += "=" * 40 + "\n\n"
        
        # Scores par technique
        for technique, data in results.items():
            report += f"\n{technique.upper()}\n"
            report += f"Score moyen : {data['average']:.1f}/10\n"
            for metric, score in data['scores'].items():
                report += f"  - {metric}: {score:.1f}/10\n"
        
        # Meilleure technique par métrique
        report += "\n\n🏆 MEILLEURES TECHNIQUES PAR CRITÈRE\n"
        for metric in self.metrics.keys():
            best_technique = max(results.items(), 
                               key=lambda x: x[1]['scores'].get(metric, 0))[0]
            best_score = results[best_technique]['scores'].get(metric, 0)
            report += f"- {metric}: {best_technique} ({best_score:.1f}/10)\n"
        
        return report

### 🏁 Test comparatif final

In [ ]:
# Problème test pour comparaison
test_problem = """
Comment une PME de 50 personnes peut-elle implémenter une culture d'innovation 
sans perturber ses opérations quotidiennes ?
"""

# Générer des réponses avec différentes techniques
print("🧪 Génération des réponses avec différentes techniques...")

responses = {}

# 1. Approche directe
print("\n1️⃣ Approche directe...")
try:
    direct = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": test_problem}],
        temperature=0.7
    )
    responses['direct'] = direct.choices[0].message.content
except:
    responses['direct'] = "Erreur"

# 2. Tree-of-Thoughts
print("\n2️⃣ Tree-of-Thoughts...")
tot_instance = TreeOfThoughts(branches=2, depth=2)
tot_result = tot_instance.solve(test_problem, ["Faisabilité", "Impact", "Coût"])
if tot_result['best_path']:
    responses['tree_of_thoughts'] = "\n".join([n.content for n in tot_result['best_path']])

# 3. Self-Consistency
print("\n3️⃣ Self-Consistency...")
sc_instance = SelfConsistency(samples=3)
sc_result = sc_instance.get_consensus(test_problem)
if 'consensus' in sc_result:
    responses['self_consistency'] = sc_result['consensus']

# 4. Step-Back
print("\n4️⃣ Step-Back Prompting...")
sb_instance = StepBackPrompting()
sb_result = sb_instance.solve_with_stepback(test_problem)
if 'solution' in sb_result:
    responses['step_back'] = sb_result['solution']

# Évaluer et comparer
print("\n🔬 Évaluation comparative...")
evaluator = ReasoningEvaluator()
comparison = evaluator.compare_techniques(test_problem, responses)

# Afficher le rapport
print("\n" + comparison['report'])
print(f"\n🏆 Technique gagnante : {comparison['winner'].upper()}")

## 📚 Récapitulatif et Bonnes Pratiques

### 🎯 Quand utiliser chaque technique :

#### Tree-of-Thoughts (ToT)
- ✅ Problèmes avec multiples solutions possibles
- ✅ Besoin d'exploration créative
- ✅ Décisions stratégiques complexes
- ❌ Éviter pour les questions simples (overhead important)

#### Self-Consistency
- ✅ Besoin de haute fiabilité
- ✅ Questions critiques nécessitant validation
- ✅ Réduction du bruit et des erreurs
- ❌ Éviter si le temps/coût est critique

#### Step-Back Prompting
- ✅ Problèmes nécessitant compréhension profonde
- ✅ Cas où les principes fondamentaux sont importants
- ✅ Transfert de connaissances entre domaines
- ❌ Éviter pour les tâches purement exécutives

### 💡 Conseils d'implémentation :

1. **Commencez simple** : Testez d'abord chaque technique individuellement
2. **Mesurez l'impact** : Utilisez des métriques pour valider l'amélioration
3. **Adaptez au contexte** : Chaque domaine a ses spécificités
4. **Optimisez les coûts** : Ces techniques consomment plus de tokens
5. **Documentez** : Gardez trace des configurations qui fonctionnent

## 🚀 Exercices Pratiques

### Exercice 1 : Implémenter votre propre variante de ToT

Créez une version de Tree-of-Thoughts qui :
- Utilise différents modèles pour chaque branche
- Implémente un système de vote entre branches
- Permet le backtracking si une branche échoue

In [ ]:
# Votre code ici
# Indice : Commencez par modifier la classe TreeOfThoughts
# pour supporter multiple modèles

### Exercice 2 : Créer un benchmark de raisonnement

Développez un ensemble de problèmes tests pour évaluer systématiquement les différentes techniques :

In [ ]:
# Créez un benchmark avec différents types de problèmes
benchmark_problems = [
    {
        'type': 'analytical',
        'problem': 'Comment optimiser la supply chain d\'une entreprise e-commerce ?',
        'expected_qualities': ['structured', 'data-driven', 'actionable']
    },
    # Ajoutez plus de problèmes...
]

# Votre code pour tester systématiquement

## 🎓 Pour aller plus loin

### 📚 Ressources recommandées :
- [Paper: Tree of Thoughts: Deliberate Problem Solving with Large Language Models](https://arxiv.org/abs/2305.10601)
- [Paper: Self-Consistency Improves Chain of Thought Reasoning](https://arxiv.org/abs/2203.11171)
- [Paper: Take a Step Back: Evoking Reasoning via Abstraction](https://arxiv.org/abs/2310.06117)

### 🔬 Prochaines étapes :
1. Expérimentez avec des combinaisons personnalisées
2. Créez des pipelines de raisonnement pour votre domaine
3. Mesurez quantitativement les améliorations
4. Partagez vos découvertes avec la communauté !

---

**Prochain notebook** : 03_Optimisation_Evaluation.ipynb - Métriques et optimisation automatique des prompts